In [1]:
import numpy as np
import dask.array as da

x = np.arange(10)
y = da.from_array(x, chunks=5)
y.compute()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
#calculate mean
x = np.arange(1000)
y = da.from_array(x, chunks=(100))
y.mean().compute()

499.5

In [10]:
#trying dask on adding and sleeping
from time import sleep
def inc(x):
    sleep(1)
    return x+1

def add(x,y):
    sleep(1)
    return x+y


x = inc(1)
y = inc(2)
z = add(x,y)
z

5

In [12]:
#trying dask on adding and sleeping
import dask
from time import sleep
@dask.delayed
def inc(x):
    sleep(1)
    return x+1

@dask.delayed
def add(x,y):
    sleep(1)
    return x+y


x = inc(1)
y = inc(2)
z = add(x,y)
z.compute()

5

In [20]:
#parallise a for loop
data = np.arange(1,8)

def inc(x):
    sleep(1)
    return x+ 1

results = []
for x in data:
    y = inc(x)
    results.append(y)

total = sum(results)
total


35

In [22]:
#parallise a for loop
data = np.arange(1,8)

@dask.delayed
def inc(x):
    sleep(1)
    return x+ 1

results = []
for x in data:
    y = inc(x)
    results.append(y)

total = sum(results)

total.compute()

35

In [24]:
import dask
from dask import delayed
from shapely.geometry import Polygon
from shapely.ops import unary_union

# Define a function to calculate the intersection between two polygons
def intersect_polygons(poly1, poly2):
    return poly1.intersection(poly2).buffer(0)

# Generate a list of polygons (replace this with your data)
polygons = [
    Polygon([(0, 0), (0, 1), (1, 1), (1, 0)]),
    Polygon([(0.5, 0.5), (0.5, 1.5), (1.5, 1.5), (1.5, 0.5)]),
    Polygon([(1, 1), (1, 2), (2, 2), (2, 1)]),
    # Add more polygons here as needed
]

# Use Dask to parallelize the intersection process
def parallel_intersection(polygons):
    tasks = []
    num_polygons = len(polygons)

    for i in range(num_polygons):
        for j in range(i + 1, num_polygons):
            # Create delayed tasks for each polygon pair intersection
            task = delayed(intersect_polygons)(polygons[i], polygons[j])
            tasks.append(task)

    # Compute the intersections in parallel
    return dask.compute(*tasks)

# Perform the parallel intersection
intersections = parallel_intersection(polygons)
intersections


(<POLYGON ((1 1, 1 0.5, 0.5 0.5, 0.5 1, 1 1))>,
 <POLYGON EMPTY>,
 <POLYGON ((1.5 1.5, 1.5 1, 1 1, 1 1.5, 1.5 1.5))>)